In [42]:
import weaviate
import json
import weaviate
import weaviate.classes as wvc
import os

import langchain
import cohere
from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import Weaviate
from langchain.schema.document import Document

from dotenv import load_dotenv

load_dotenv()
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

# client = weaviate.Client(
#     url = "http://localhost:8080",  # Replace with your endpoint
#     additional_headers = {
#         "X-Cohere-Api-Key": COHERE_API_KEY,
#     }
# )
# print("Weaviate version:", weaviate.__version__)

In [43]:
print(COHERE_API_KEY)

pCloNmMvKU5moXPFM9ZhfLESO1il5E2bpzvGUTaX


In [44]:
client = weaviate.connect_to_local(port=8080, 
                                   grpc_port=50051,
                                   headers={
                                              "X-Cohere-Api-Key": COHERE_API_KEY,
 
                                   })



In [45]:

if (client.collections.exists("Article")):
  # delete collection "Article" - THIS WILL DELETE THE COLLECTION AND ALL ITS DATA
  client.collections.delete("Article")  # Replace with your collection name


embeddings = CohereEmbeddings(cohere_api_key=COHERE_API_KEY, model="embed-english-v3.0")
test_documents = [
    "Rust is the most loved programming language",
    "Python is an interpreted programming language", 
    "Pythons are reptiles", 
    "programming is a lucrative career in the 21st century"
]

In [46]:

embeddings = CohereEmbeddings(cohere_api_key=COHERE_API_KEY, model="embed-english-v3.0")
test_documents = [
    "Rust is the most loved programming language",
    "Python is an interpreted programming language", 
    "Pythons are reptiles", 
    "programming is a lucrative career in the 21st century"
]

'''client = weaviate.Client(
    embedded_options=weaviate.EmbeddedOptions(port=8080),
    additional_headers={
        "X-Cohere-Api-Key": COHERE_API_KEY,
    }
)'''

'client = weaviate.Client(\n    embedded_options=weaviate.EmbeddedOptions(port=8080),\n    additional_headers={\n        "X-Cohere-Api-Key": COHERE_API_KEY,\n    }\n)'

In [47]:
if (client.collections.exists("Test_Database")):
  # delete collection "Article" - THIS WILL DELETE THE COLLECTION AND ALL ITS DATA
  client.collections.delete("Test_Database")  # Replace with your collection name



In [48]:
vectorstore = client.collections.create(
    name="Test_Database",
    vectorizer_config=wvc.Configure.Vectorizer.text2vec_cohere(),  # If set to "none" you must always provide vectors yourself. Could be any other "text2vec-*" also.
    generative_config=wvc.Configure.Generative.cohere()  # Ensure the `generative-openai` module is used for generative queries
)

In [49]:
document_embeddings = embeddings.embed_documents(test_documents)

objs = []
for i, (d,e) in enumerate(zip(test_documents, document_embeddings)):
    objs.append(wvc.DataObject(
        properties={
            "text": d,
            "index": i,
        },
        vector=e
    ))


In [50]:
vectorstore = client.collections.get("Test_Database")

vectorstore.data.insert_many(objs)    # This uses batching under the hood


BatchObjectReturn(all_responses=[UUID('65a23d3d-9685-4db7-b691-b046c9ac69ab'), UUID('9580e0c3-86bc-4538-b6c3-c7c995c6ea3d'), UUID('14397363-fbd2-4111-9483-398bb74405cb'), UUID('25784f11-31cc-420d-a5b6-d3612e1842c3')], uuids={0: UUID('65a23d3d-9685-4db7-b691-b046c9ac69ab'), 1: UUID('9580e0c3-86bc-4538-b6c3-c7c995c6ea3d'), 2: UUID('14397363-fbd2-4111-9483-398bb74405cb'), 3: UUID('25784f11-31cc-420d-a5b6-d3612e1842c3')}, errors={}, elapsed_seconds=0.005797863006591797, has_errors=False)

In [51]:
response = vectorstore.query.near_text(
    query="snake",
    limit=2
)

WeaviateQueryException: Query call failed with message explorer: get class: vectorize params: vectorize params: vectorize params: vectorize keywords: remote client vectorize: Cohere API Key: no api key found neither in request header: X-Cohere-Api-Key nor in environment variable under COHERE_APIKEY.

In [24]:

#vectorstore = client.collections.get("Question")
vectorstore.data.insert_many(objs)    # This uses batching under the hood

response = vectorstore.query.near_text(
    query="snake",
    limit=2
)

print(response.objects[0].properties)  # Inspect the first object



# using langchain
'''documents = []
for i, test_doc in enumerate(test_documents): 
    vstore_doc = Document(
                page_content=test_doc,
                metadata={
                    'source': str(i)
                }, 
            )

db = Weaviate.from_documents(documents, embeddings, index_name="WikipediaLangChain", client=client, by_text=False)
docs = db.similarity_search("is python compiled or interpreted?")
print(docs)'''



WeaviateQueryException: Query call failed with message panic occurred: ValidateParam was called without any known params present.